In [1]:
import numpy as np
import tensorflow as tf
import gym
import os
import datetime
from statistics import mean
from gym import wrappers

In [2]:
env = gym.make('CartPole-v0')
ddd=5
# len(env.observation_space.sample())

In [3]:
from Sarsa import SARSA
Agent = SARSA(input_dimention = (4), num_actions = env.action_space.n,
            gamma = 0.99, max_experiences = 1000, min_experiences = 100,
            batch_size = 32,
            lr = 0.01)

In [4]:
def play_game(env, Agent, epsilon):
    rewards = 0
    iter = 0
    done = False
    losses = list()
    
    prev_observations = env.reset()
    
    prev_action = Agent.get_action(prev_observations, epsilon)
    
    while not done:
        
#         prev_observations = observations
        n_observations, reward, done, _ = env.step(prev_action)
        n_action = Agent.get_action(n_observations, epsilon)

        exp = {'s': prev_observations, 'a': prev_action, 'r': reward, 's2': n_observations,  'a2': n_action,'done': done}
        Agent.add_experience(exp)
        loss = Agent.train_pop()
        
        rewards += reward
        
        prev_observations = n_observations
        prev_action = n_action
        
        if done:
            reward = -200
            prev_observations = env.reset()
            prev_action = Agent.get_action(prev_observations, epsilon)
        
        if isinstance(loss, int):
            losses.append(loss)
        else:
            losses.append(loss.numpy())
        iter += 1

    return rewards, mean(losses)

In [5]:
def make_video(env, Agent):
    env = wrappers.Monitor(env, os.path.join(os.getcwd(), "Sarsa_video"), force=True)
    rewards = 0
    steps = 0
    done = False
    observation = env.reset()
    while not done:
        env.render()
        action = Agent.get_action(observation, 0)
        observation, reward, done, _ = env.step(action)
        steps += 1
        rewards += reward
    print("Testing steps: {} rewards {}: ".format(steps, rewards))


In [ ]:
# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# log_dir = 'logs/Sarsa/' + current_time
# summary_writer = tf.summary.create_file_writer(log_dir)

ep=[]
ep_r=[]
eps=[]
avg_r=[]
ep_l=[]

N = 5000
total_rewards = np.empty(N)
epsilon = 1.
decay = 0.99
min_epsilon = 0.01
for n in range(N):
    epsilon = max(min_epsilon, epsilon * decay)
    total_reward, losses = play_game(env, Agent, epsilon)
    total_rewards[n] = total_reward
    avg_rewards = total_rewards[max(0, n - 100):(n + 1)].mean()
   
#     with summary_writer.as_default():
#         tf.summary.scalar('episode reward', total_reward, step=n)
#         tf.summary.scalar('running avg reward(100)', avg_rewards, step=n)
#         tf.summary.scalar('average loss)', losses, step=n)
    if n % 100 == 0:
        
        ep.append(n)
        ep_r.append(total_reward)
        eps.append(epsilon)
        avg_r.append(avg_rewards)
        ep_l.append(losses)
        
        print("episode:", n, "episode reward:", total_reward, "eps:", epsilon, "avg reward (last 100):", avg_rewards,
              "episode loss: ", losses)
print("avg reward for last 100 episodes:", avg_rewards)
make_video(env, Agent)
env.close()

episode: 0 episode reward: 47.0 eps: 0.99 avg reward (last 100): 47.0 episode loss:  0
episode: 100 episode reward: 9.0 eps: 0.36237201786049694 avg reward (last 100): 14.623762376237623 episode loss:  0.12274975
episode: 200 episode reward: 10.0 eps: 0.13263987810938213 avg reward (last 100): 10.752475247524753 episode loss:  0
episode: 300 episode reward: 8.0 eps: 0.0485504851305729 avg reward (last 100): 9.732673267326733 episode loss:  0.51424694
episode: 400 episode reward: 8.0 eps: 0.017771047742294682 avg reward (last 100): 9.455445544554456 episode loss:  0
episode: 500 episode reward: 9.0 eps: 0.01 avg reward (last 100): 9.613861386138614 episode loss:  0
episode: 600 episode reward: 12.0 eps: 0.01 avg reward (last 100): 10.158415841584159 episode loss:  0
episode: 700 episode reward: 11.0 eps: 0.01 avg reward (last 100): 10.257425742574258 episode loss:  0
episode: 800 episode reward: 13.0 eps: 0.01 avg reward (last 100): 12.257425742574258 episode loss:  0
episode: 900 episo

In [ ]:
# result = np.asarray(
#          [ep,
#          ep_r,
#          eps,
#          avg_r,
#          ep_l])
# np.savetxt('resalt_SARSA', result)